<a href="https://colab.research.google.com/github/qsardor/GoogleColabProjects/blob/main/Tile_Upscaler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **Launch Tile-Upscaler**
#@markdown This cell will clone the repository, modify the launch settings for a public URL, install dependencies quietly, and start the Gradio app. Please wait for the installation and launch process to complete. A public URL (like `*.gradio.live`) will appear in the output below.

import os
import sys
import re # Import regex module for more flexible replacement

print("⏳ Cloning repository...")
# Clone quietly
!git clone -q https://huggingface.co/spaces/gokaygokay/Tile-Upscaler app
print("✅ Repository cloned.")

# Change directory
try:
    os.chdir('app')
    print(f"✅ Changed directory to: {os.getcwd()}")
except FileNotFoundError:
    print(f"❌ Error: 'app' directory not found after cloning. Check clone step.")
    sys.exit(1)

print("⏳ Modifying app.py for public sharing...")
# Modify app.py to add share=True using Python
try:
    with open('app.py', 'r', encoding='utf-8') as f:
        content = f.read()

    # Use regex to find .launch() potentially with arguments inside and add share=True
    # This is more robust than a simple string replace
    # It looks for patterns like app.launch(...), demo.launch(), iface.launch(debug=True), etc.
    modified_content, n_replacements = re.subn(
        r"(\.launch\s*\()\s*([^)]*\)?)",  # Pattern: .launch( possibly_existing_args )
        r'\1share=True, \2',          # Replacement: .launch(share=True, possibly_existing_args )
        content,
        count=1                       # Replace only the first occurrence towards the end likely
    )

    if n_replacements > 0:
        with open('app.py', 'w', encoding='utf-8') as f:
            f.write(modified_content)
        print("✅ app.py modified successfully to include share=True.")
        # Optional: verify the change
        # !grep "launch(share=True" app.py
    else:
        # Fallback or warning if the pattern wasn't found
        print("⚠️ Warning: Could not automatically find and modify '.launch()' in app.py.")
        print("   The app might launch without a public URL. Trying simple replacement as fallback...")
        # Try simple replacement as a fallback (less robust)
        if 'demo.launch()' in content:
             modified_content = content.replace('demo.launch()', 'demo.launch(share=True)')
             with open('app.py', 'w', encoding='utf-8') as f:
                 f.write(modified_content)
             print("✅ app.py modified using simple replacement.")
        elif 'app.launch()' in content:
             modified_content = content.replace('app.launch()', 'app.launch(share=True)')
             with open('app.py', 'w', encoding='utf-8') as f:
                 f.write(modified_content)
             print("✅ app.py modified using simple replacement.")
        else:
             print("❌ Fallback replacement failed. Manual check of app.py might be needed if public URL is required.")
             # Proceeding anyway, might only get local URL

except FileNotFoundError:
    print(f"❌ Error: 'app.py' not found in the {os.getcwd()} directory.")
    sys.exit(1)
except Exception as e:
    print(f"❌ Error modifying app.py: {e}")
    # Decide if you want to exit or continue without the modification
    # sys.exit(1) # Or just print a warning and continue

print("⏳ Installing dependencies quietly... (This may take a few minutes)")
# Install quietly and redirect output
!pip install --quiet -r requirements.txt > pip_install.log 2>&1
# Check if installation seems okay by checking for a common package (e.g., gradio)
try:
    import gradio
    print("✅ Dependencies installed successfully.")
except ImportError:
    print("⚠️ Warning: Dependency installation might have failed or Gradio is not directly imported.")
    print("   Check pip_install.log for details if errors occur.")
    # !cat pip_install.log # Uncomment to view log if needed

print("\n🚀 Launching Gradio app... Please wait for the public URL.")
# Run the app - Output will include the Gradio link
!python app.py